In [1]:
from typing import Union

from langchain_google_vertexai import VertexAI

In [2]:
def object_to_xml(data: Union[dict, bool], root='object'):
    xml = f'<{root}>'
    if isinstance(data, dict):
        for key, value in data.items():
            xml += object_to_xml(value, key)

    elif isinstance(data, (list, tuple, set)):
        for item in data:
            xml += object_to_xml(item, 'item')

    else:
        xml += str(data)

    xml += f'</{root}>'
    return xml

In [3]:
tasks = [
    {'due':'03/09/2024', 'description': 'submit summer work to physics', 'status': 100},
    {'due':'05/09/2024', 'description': 'request move change to AP US History', 'status':0}
]

In [4]:
object_to_xml(tasks, 'tasks')

'<tasks><item><due>03/09/2024</due><description>submit summer work to physics</description><status>100</status></item><item><due>05/09/2024</due><description>request move change to AP US History</description><status>0</status></item></tasks>'

In [5]:
model = VertexAI(model='gemini-1.5-pro-002')

In [8]:
model.invoke(f'Generate potential history of notes from high school student about his tasks. The notes should lead to list of tasks as follows {tasks}. Output format date(dd/mm); note')

"29/08; Need to finish summer work for physics. Due next week.\n30/08; Physics work done! Just need to submit it.\n01/09; Submit physics work tomorrow.\n03/09; Submitted physics summer work.\n03/09; Really not enjoying regular US History.  Need to see about switching to AP.\n03/09; Ask counselor about switching to APUSH.  Maybe tomorrow?\n04/09; Totally forgot to talk to counselor. Do it tomorrow for sure.\n\n```json\n[{'due': '03/09/2024', 'description': 'submit summer work to physics', 'status': 100}, {'due': '05/09/2024', 'description': 'request move change to AP US History', 'status': 0}]\n```\n"